In [1]:
import pandas as pd
import numpy as np
import os

# Import Data

In [2]:
# set the path of the processed data

processed_data_path = os.path.join(os.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [3]:
train_df = pd.read_csv(train_file_path, index_col='PassengerId')
test_df = pd.read_csv(test_file_path, index_col='PassengerId')

In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
Age                   418 non-null float64
Fare                  418 non-null float64
FamilySize            418 non-null int64
IsMother              418 non-null int64
IsMale                418 non-null int64
Deck_A                418 non-null int64
Deck_B                418 non-null int64
Deck_C                418 non-null int64
Deck_D                418 non-null int64
Deck_E                418 non-null int64
Deck_F                418 non-null int64
Deck_G                418 non-null int64
Deck_Z                418 non-null int64
Pclass_1              418 non-null int64
Pclass_2              418 non-null int64
Pclass_3              418 non-null int64
Title_Lady            418 non-null int64
Title_Master          418 non-null int64
Title_Miss            418 non-null int64
Title_Mr              418 non-null int64
Title_Mrs             418 non-null int64
Title_Officer         418 n

## Data Preparation

In [5]:
X = train_df.loc[:, 'Age': ].values.astype('float')
y = train_df['Survived'].ravel()

In [6]:
print(X.shape,y.shape)

(891, 32) (891,)


In [7]:
# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [8]:
# average survival in train and test
print('mean survival in train: {0:3f}'.format(np.mean(y_train)))
print('mean survival in test: {0:3f}'.format(np.mean(y_test)))

mean survival in train: 0.383427
mean survival in test: 0.385475


### Check Scikit-Learn version

In [9]:
import sklearn

In [10]:
sklearn.__version__ # making sure we have at least version 0.19 (for DummyClassifier) - else we need to update

'0.19.1'

## Baseline Model

In [11]:
from sklearn.dummy import DummyClassifier

In [12]:
# create model
model_dummy = DummyClassifier(strategy = 'most_frequent', random_state=0)

In [13]:
# train model
model_dummy.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [14]:
#  evaluate the model with the test data
print('score for baseline model: {0:.3f}'.format(model_dummy.score(X_test, y_test)))

score for baseline model: 0.615


In [15]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [16]:
# accuracy score
print('accuracy for baseline model: {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

accuracy for baseline model: 0.61


In [17]:
# confusion matrix
print('confusion matrix for baseline model: \n {}'.format(confusion_matrix(y_test, model_dummy.predict(X_test))))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [18]:
# precision and recall
print('precision for baseline model: {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('recall for baseline model: {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

precision for baseline model: 0.00
recall for baseline model: 0.00


/home/sgrosu/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


# First Kaggle submission

In [19]:
# converting to matrix
test_X = test_df.values.astype('float')

In [20]:
# make predictions
predictions = model_dummy.predict(test_X)

In [21]:
df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})

In [22]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [23]:
submission_data_path = os.path.join(os.pardir, 'data','external')
submission_file_path = os.path.join(submission_data_path, '01_dummy.csv')

In [24]:
df_submission.to_csv(submission_file_path, index=False)

In [25]:
# create function for reproducible analysis
def get_submission_file(model, filename):
    # converting to matrix
    test_X = test_df.values.astype('float')
    # make predictions
    predictions = model.predict(test_X)
    # create submission dataframe
    df_submission = pd.DataFrame({'PassengerId': test_df.index, 'Survived': predictions})
    # creating paths
    submission_data_path = os.path.join(os.pardir, 'data','external')
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to file
    df_submission.to_csv(submission_file_path, index=False)

In [26]:
# get the submission file via function
get_submission_file(model_dummy, '01_dummy.csv')

# Logistic Regression Model

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
# create model
model_lr_1 = LogisticRegression(random_state=0)

In [30]:
# train model
model_lr_1.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [31]:
# evaluate model
print('score for logistic regression - version 1: {0:.2f}'.format(model_lr_1.score(X_test, y_test)))

score for logistic regression - version 1: 0.83


In [37]:
# performance metrics
# accuracy
print('accuracy:', accuracy_score(y_test, model_lr_1.predict(X_test)))
# confusion matrix
print('confusion matrix: \n', confusion_matrix(y_test, model_lr_1.predict(X_test)))
print('precision:', precibsion_score(y_test, model_lr_1.predict(X_test)))
print('recall:', recall_score(y_test, model_lr_1.predict(X_test)))

accuracy: 0.8324022346368715
confusion matrix: 
 [[95 15]
 [15 54]]
precision: 0.782608695652174
recall: 0.782608695652174


In [38]:
# model coefficients
model_lr_1.coef_

array([[-0.02753556,  0.00455148, -0.46450195,  0.        , -0.88164239,
         0.13225315, -0.16691832, -0.41245506,  0.49317162,  1.10319296,
         0.38640447, -0.13092518, -0.30709503,  0.94996091,  0.49542732,
        -0.34775963,  0.2605417 ,  1.17167116,  0.47968949, -1.4445742 ,
         1.2390398 , -0.12612194, -0.48261741,  0.17297391,  0.24425427,
         0.27829476,  0.40210566,  0.48997874,  0.4664344 ,  0.14121546,
         0.39167748,  0.70595112]])

# Second Kaggle Submission

In [39]:
# get submission file
get_submission_file(model_lr_1, '02_lr.csv')